In [ ]:
import pandas as pd
import json
import glob


In [ ]:
json_files = glob.glob('/content/drive/MyDrive/data/*.json')
data = []

for file in json_files:
    with open(file, 'r') as f:
        data.append(json.load(f))

df = pd.json_normalize(data)


In [ ]:
sensors_df = df.explode('sensors')

mean_sensor_df = pd.DataFrame()

mean_sensor_df['OccupantID'] = sensors_df['OccupantID']
mean_sensor_df['SeatID'] = sensors_df['SeatID']
mean_sensor_df['out_of_position'] = sensors_df['sensors'].apply(lambda x: x['out_of_position'])
mean_sensor_df['timestamp'] = sensors_df['sensors'].apply(lambda x: x['timestamp'])

for i in range(1, 11):
    mean_sensor_df[f'Sensor {i}'] = sensors_df['sensors'].apply(lambda x: abs(x[f'Sensor {i}']))

mean_sensor_df['timestamp'] = pd.to_datetime(mean_sensor_df['timestamp'], format='%H:%M:%S')
mean_sensor_df.drop(columns=['timestamp'], inplace=True, errors='ignore')
mean_sensor_df

,OccupantID,SeatID,out_of_position,Sensor 1,Sensor 2,Sensor 3,Sensor 4,Sensor 5,Sensor 6,Sensor 7,Sensor 8,Sensor 9,Sensor 10
0,1,P8,Nominal position,76265.00,78556.43,130601.36,108138.43,107610.81,82878.83,297.14,2047.15,102875.56,133844.22
0,1,P8,Nominal position,75979.28,81316.44,131563.26,108375.57,102107.94,81366.44,303.81,1915.72,102682.22,134856.61
0,1,P8,Nominal position,75979.28,81316.44,131563.26,108375.57,102107.94,81366.44,303.81,1915.72,102682.22,134856.61
0,1,P8,Nominal position,75979.28,81316.44,131563.26,108375.57,102107.94,81366.44,303.81,1915.72,102682.22,134856.61
0,1,P8,Nominal position,75979.28,81316.44,131563.26,108375.57,102107.94,81366.44,303.81,1915.72,102682.22,134856.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,29,P8,Pelvis drift,2698.28,1656.80,52456.46,41179.07,11720.05,23953.76,119.05,2153.90,31305.30,19706.15
27,29,P8,Pelvis drift,2698.28,1656.80,52456.46,41179.07,11720.05,23953.76,119.05,2153.90,31305.30,19706.15
27,29,P8,Pelvis drift,2698.28,1656.80,52456.46,41179.07,11720.05,23953.76,119.05,2153.90,31305.30,19706.15
27,29,P8,Pelvis drift,2698.28,1656.80,52456.46,41179.07,11720.05,23953.76,119.05,2153.90,31305.30,19706.15


In [ ]:
clean_df = mean_sensor_df.drop_duplicates(subset=['Sensor 1', 'Sensor 2', 'Sensor 3', 'Sensor 4', 'Sensor 5', 'Sensor 6', 'Sensor 9', 'Sensor 10'])


In [ ]:
clean_df.drop(columns=['SeatID', 'Sensor 7', 'Sensor 8'], inplace=True)
clean_df

<ipython-input-5-502fa3b2c2e3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df.drop(columns=['SeatID', 'Sensor 7', 'Sensor 8'], inplace=True)


,OccupantID,out_of_position,Sensor 1,Sensor 2,Sensor 3,Sensor 4,Sensor 5,Sensor 6,Sensor 9,Sensor 10
0,1,Nominal position,76265.00,78556.43,130601.36,108138.43,107610.81,82878.83,102875.56,133844.22
0,1,Nominal position,75979.28,81316.44,131563.26,108375.57,102107.94,81366.44,102682.22,134856.61
0,1,Nominal position,78276.43,89382.18,132477.55,108127.95,99717.45,73190.23,104578.42,145262.35
0,1,Nominal position,79017.39,89050.75,132420.41,108495.57,100113.64,73534.99,104620.32,145073.78
0,1,Nominal position,79087.86,88158.37,131935.65,108832.72,100337.45,75321.66,104813.66,144533.78
...,...,...,...,...,...,...,...,...,...,...
27,29,Pelvis drift,2896.35,1960.12,53002.15,40931.92,12753.30,19875.83,34390.13,21471.77
27,29,Pelvis drift,2886.35,1952.02,52962.62,41223.83,12797.59,20802.98,33773.46,21508.44
27,29,Pelvis drift,2663.99,1770.13,52601.69,41118.60,12216.20,24000.92,35110.48,21422.25
27,29,Pelvis drift,2491.14,1656.80,52706.93,41187.17,12094.79,25183.25,33681.44,20548.02


In [ ]:
clean_df['out_of_position'].value_counts()

,count
out_of_position,
Pelvis drift,1306
Leaning forward,1258
Dissymetry,1203
Nominal position,1202


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
clean_df['out_of_position_numeric'] = le.fit_transform(clean_df['out_of_position'])

clean_df.drop(columns=['out_of_position'], inplace=True)

<ipython-input-7-89c5f122fff2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['out_of_position_numeric'] = le.fit_transform(clean_df['out_of_position'])
<ipython-input-7-89c5f122fff2>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df.drop(columns=['out_of_position'], inplace=True)


In [ ]:
clean_df

,OccupantID,Sensor 1,Sensor 2,Sensor 3,Sensor 4,Sensor 5,Sensor 6,Sensor 9,Sensor 10,out_of_position_numeric
0,1,76265.00,78556.43,130601.36,108138.43,107610.81,82878.83,102875.56,133844.22,2
0,1,75979.28,81316.44,131563.26,108375.57,102107.94,81366.44,102682.22,134856.61,2
0,1,78276.43,89382.18,132477.55,108127.95,99717.45,73190.23,104578.42,145262.35,2
0,1,79017.39,89050.75,132420.41,108495.57,100113.64,73534.99,104620.32,145073.78,2
0,1,79087.86,88158.37,131935.65,108832.72,100337.45,75321.66,104813.66,144533.78,2
...,...,...,...,...,...,...,...,...,...,...
27,29,2896.35,1960.12,53002.15,40931.92,12753.30,19875.83,34390.13,21471.77,3
27,29,2886.35,1952.02,52962.62,41223.83,12797.59,20802.98,33773.46,21508.44,3
27,29,2663.99,1770.13,52601.69,41118.60,12216.20,24000.92,35110.48,21422.25,3
27,29,2491.14,1656.80,52706.93,41187.17,12094.79,25183.25,33681.44,20548.02,3


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
X = clean_df.drop(columns=['out_of_position_numeric','OccupantID'])  # Features (sensor values)
y = clean_df['out_of_position_numeric']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn import metrics
def modelfit(alg, dtrain, predictors, target, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(
            xgb_param,
            xgtrain,
            num_boost_round=alg.get_params()['n_estimators'],
            nfold=cv_folds,
            metrics='mlogloss',
            early_stopping_rounds=early_stopping_rounds
        )
        alg.set_params(n_estimators=cvresult.shape[0])

    alg.fit(dtrain[predictors], dtrain[target])

    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])

    print("\nModel Report")
    print("Training Accuracy : %.4g" % metrics.accuracy_score(dtrain[target].values, dtrain_predictions))
    print("Training AUC Score: %f" % metrics.roc_auc_score(dtrain[target], dtrain_predprob, multi_class='ovr'))

    return dtrain_predictions, dtrain_predprob

In [ ]:
xgb_model = xgb.XGBClassifier(
    max_depth=3,
    n_estimators=100,
    learning_rate=0.1,
    reg_alpha=0.1,
    reg_lambda=1.0,
    objective='multi:softmax',
    num_class=len(y.unique()),
    use_label_encoder=False
)

In [ ]:
train_data = pd.concat([X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)


In [ ]:
dtrain_predictions, dtrain_predprob = modelfit(xgb_model, train_data, X.columns, 'out_of_position_numeric')


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [10:58:30] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [10:58:32] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Model Report
Training Accuracy : 0.9932
Training AUC Score: 0.999932


In [ ]:
y_pred = xgb_model.predict(X_test)
y_pred_prob = xgb_model.predict_proba(X_test)

In [ ]:
print("\nTest Set Evaluation:")
print("Accuracy: %.4g" % metrics.accuracy_score(y_test, y_pred))
print("AUC Score (Test): %f" % metrics.roc_auc_score(y_test, y_pred_prob, multi_class='ovr'))


Test Set Evaluation:
Accuracy: 0.9748
AUC Score (Test): 0.997005


In [ ]:
print("\nComparison of Training and Testing Metrics:")
print("Training Accuracy: %.4g" % metrics.accuracy_score(y_train, dtrain_predictions))
print("Testing Accuracy: %.4g" % metrics.accuracy_score(y_test, y_pred))
print("Training AUC Score: %f" % metrics.roc_auc_score(y_train, dtrain_predprob, multi_class='ovr'))
print("Testing AUC Score: %f" % metrics.roc_auc_score(y_test, y_pred_prob, multi_class='ovr'))



Comparison of Training and Testing Metrics:
Training Accuracy: 0.9932
Testing Accuracy: 0.9748
Training AUC Score: 0.999932
Testing AUC Score: 0.997005


# Model testing

In [ ]:
json_files = glob.glob('/content/*.json')
data = []

for file in json_files:
    with open(file, 'r') as f:
        data.append(json.load(f))

df1 = pd.json_normalize(data)

In [ ]:
import joblib
import xgboost as xgb

# Save the LabelEncoder
joblib.dump(le, 'label_encoder.pkl')

# Save the XGBoost model
xgb_model.save_model('xgb_model.json')

print("Label Encoder and XGBoost model saved successfully.")


NameError: name 'le' is not defined

In [ ]:
sensors1= df1.explode('sensors')

df1 = pd.DataFrame()

df1['OccupantID'] = sensors1['OccupantID']
df1['SeatID'] = sensors1['SeatID']
df1['out_of_position'] = sensors1['sensors'].apply(lambda x: x['out_of_position'])
df1['timestamp'] = sensors1['sensors'].apply(lambda x: x['timestamp'])

for i in range(1, 11):
    df1[f'Sensor {i}'] = sensors1['sensors'].apply(lambda x: abs(x[f'Sensor {i}']))

df1['timestamp'] = pd.to_datetime(df1['timestamp'], format='%H:%M:%S')
df1.drop(columns=['timestamp'], inplace=True, errors='ignore')
df1

In [ ]:
clean = df1.drop_duplicates(subset=['Sensor 1', 'Sensor 2', 'Sensor 3', 'Sensor 4', 'Sensor 5', 'Sensor 6', 'Sensor 9', 'Sensor 10'])
clean

,OccupantID,SeatID,out_of_position,Sensor 1,Sensor 2,Sensor 3,Sensor 4,Sensor 5,Sensor 6,Sensor 7,Sensor 8,Sensor 9,Sensor 10
0,2,P8,Nominal position,70384.98,95893.63,133600.89,111319.39,108794.63,88580.75,320.95,1295.24,105.71,174323.41
0,2,P8,Nominal position,70275.46,95929.82,133716.13,111347.96,109269.39,88531.23,320.95,1284.77,113.33,174839.60
0,2,P8,Nominal position,70179.74,96001.73,133714.70,111247.96,109237.49,88989.33,333.33,1326.20,124.76,174194.83
0,2,P8,Nominal position,70097.36,96078.40,133776.61,111247.96,109247.48,89019.80,333.33,1360.96,136.19,174352.93
0,2,P8,Nominal position,70045.46,96147.92,133723.75,111416.54,109167.96,89008.37,313.33,1393.82,128.57,174062.45
0,2,P8,Nominal position,69917.84,96170.30,133778.51,111387.01,109002.72,89000.76,318.10,1404.77,113.33,174097.21
0,2,P8,Nominal position,69834.03,96118.87,133838.99,111407.01,108940.81,89132.18,313.33,1456.67,113.33,174425.30
0,2,P8,Nominal position,69990.22,96080.78,133875.65,111385.11,108839.87,89341.71,344.76,1386.20,120.95,174272.93
0,2,P8,Nominal position,69961.65,96102.68,133908.51,111365.11,108750.82,89263.61,332.38,1415.24,148.57,174435.78
0,2,P8,Nominal position,70024.98,95970.77,133926.61,111414.63,108707.96,89259.80,344.76,1410.01,136.19,174661.50


In [ ]:
clean.drop(columns=['SeatID', 'Sensor 7', 'Sensor 8','OccupantID'], inplace=True)
clean

<ipython-input-22-3a1e63b6841b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean.drop(columns=['SeatID', 'Sensor 7', 'Sensor 8','OccupantID'], inplace=True)


,out_of_position,Sensor 1,Sensor 2,Sensor 3,Sensor 4,Sensor 5,Sensor 6,Sensor 9,Sensor 10
0,Nominal position,70384.98,95893.63,133600.89,111319.39,108794.63,88580.75,105.71,174323.41
0,Nominal position,70275.46,95929.82,133716.13,111347.96,109269.39,88531.23,113.33,174839.60
0,Nominal position,70179.74,96001.73,133714.70,111247.96,109237.49,88989.33,124.76,174194.83
0,Nominal position,70097.36,96078.40,133776.61,111247.96,109247.48,89019.80,136.19,174352.93
0,Nominal position,70045.46,96147.92,133723.75,111416.54,109167.96,89008.37,128.57,174062.45
0,Nominal position,69917.84,96170.30,133778.51,111387.01,109002.72,89000.76,113.33,174097.21
0,Nominal position,69834.03,96118.87,133838.99,111407.01,108940.81,89132.18,113.33,174425.30
0,Nominal position,69990.22,96080.78,133875.65,111385.11,108839.87,89341.71,120.95,174272.93
0,Nominal position,69961.65,96102.68,133908.51,111365.11,108750.82,89263.61,148.57,174435.78
0,Nominal position,70024.98,95970.77,133926.61,111414.63,108707.96,89259.80,136.19,174661.50


In [ ]:
clean1 = clean.copy()

In [ ]:
clean1 = clean1.drop(columns=['out_of_position'])


In [ ]:
import xgboost as xgb
xgb_model_loaded = xgb.XGBClassifier()
xgb_model_loaded.load_model('/content/drive/MyDrive/model/xgb_model.json')

In [ ]:
predictions = xgb_model_loaded.predict(clean1)


In [ ]:
clean['Predictions'] = predictions

<ipython-input-27-beaded4887e2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean['Predictions'] = predictions


In [ ]:
import joblib
le_loaded = joblib.load('label_encoder.pkl')


In [ ]:
from sklearn.preprocessing import LabelEncoder

clean['Predictions'] = le_loaded.inverse_transform(clean['Predictions'])


<ipython-input-29-7423ac7937dc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean['Predictions'] = le_loaded.inverse_transform(clean['Predictions'])


In [ ]:
pd.set_option('display.max_rows', None)

clean[['out_of_position', 'Predictions']]

,out_of_position,Predictions
0,Nominal position,Nominal position
0,Nominal position,Nominal position
0,Nominal position,Nominal position
0,Nominal position,Nominal position
0,Nominal position,Nominal position
0,Nominal position,Nominal position
0,Nominal position,Nominal position
0,Nominal position,Nominal position
0,Nominal position,Nominal position
0,Nominal position,Nominal position


In [ ]:
correct_predictions = clean['out_of_position'] == clean['Predictions']

accuracy = correct_predictions.mean()

print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 94.48%
